In [131]:
import numpy as np
from numpy import linalg
from sympy import *

In [132]:
# Rossenbrock function
# f1(x) = 10*(x2 - x1^2)
# f2(x) = 1 - x1
# x0 = (-1.2, 1)
# f = 0 at (1, 1)
# n = 2, m = 2 13

In [133]:
# Freudenstein and Roth function
# f1(x) = -13 + x1 + ((5 - x2)*x2 - 2)*x2
# f2(x) = -29 + x1 + ((x2 + 1)*x2 - 14)*x2
# x0 = (0.5, -2)
# f = 0 at (5, 4)

In [134]:
# Powell badly scaled function
# f1(x) = 10**4*x1*x2 - 1
# f2(x) = exp(-x1) + exp(-x2) - 1.0001
# x0 = (0, 1)
# f = 0 at (1.098...10**-5, 9.106...)

In [135]:
# Powell singular function
# f1(x) = x1 + 10*x2
# f2(x) = 5^(1/2)*(x3 - x4)
# f3(x) = (x2 - 2*x3)^2
# f4(x) = 10^(1/2)*(x1 - x4)^2
# x0 = [3, -1, 0, 1]
# f = 0

In [136]:
x1 = symbols('x1')
x2 = symbols('x2')
x3 = symbols('x3')
x4 = symbols('x4')

# f1 = simplify("10*(x2 - x1^2)")
# f2 = sympify("1 - x1")
# x0 = np.array([-3, 2], dtype = 'float')

# f1 = simplify("(10**4)*x1*x2 - 1")
# f2 = simplify("exp(-x1) + exp(-x2) - 1.0001")
# x0 = np.array([0, 1], dtype = 'float')

# f1 = simplify("-13 + x1 + ((5 - x2)*x2 - 2)*x2")
# f2 = simplify("-29 + x1 + ((x2 + 1)*x2 - 14)*x2")
# x0 = np.array([0.5, -2], dtype = 'float')

f1 = simplify('x1 + 10*x2')
f2 = simplify('5^(1/2)*(x3 - x4)')
f3 = simplify('(x2 - 2*x3)^2')
f4 = simplify('10^(1/2)*(x1 - x4)^2')
x0 = np.array([3, -1, 0, 1], dtype = 'float')

# x = np.array([x1, x2])
# F = np.array([f1, f2])

x = np.array([x1, x2, x3, x4])
F = np.array([f1, f2, f3, f4])

In [137]:
# n = int(input("Enter n = "))
# x = np.array([symbols('x{0}'.format(i + 1)) for i in range(n)])

# F = np.array([simplify(input("f{0} = ".format(i + 1))) for i in range(n)])
# x0 = np.array([float(input("{0} Element of x0".format(i + 1))) for i in range(n)], dtype = 'float')

eps = 10 ** -7

In [138]:
F_0 = lambda x0: np.array([F[i].subs(dict(zip(x, x0))) for i in range(len(F))], dtype = 'float')

In [139]:
def F_der_0(x0):
    F_der = np.array([[diff(F[i], x[j]) for j in range(len(x))] for i in range(len(F))])
    return np.array([[F_der[i][j].subs(dict(zip(x, x0))) for j in range(len(F_der[i]))] for i in range(len(F_der))], dtype = 'float')

In [140]:
# test function for F(x, y)
# A = np.array([simplify("x1**2 + 7*x1 + 2*x1*x2 - 3"), simplify("x1 + x2**3")])
# A = np.array([simplify("-13 + x1 + ((5 - x2)*x2 - 2)*x2"), simplify("-29 + x1 + ((x2 + 1)*x2 - 14)*x2")])
# A[0].subs({x1: 1.13975640, x2: 1.05684169})

In [141]:
def F_1(x1, x2):
    assert(len(x1) == len(x2))
    arr = []
    for i in range(len(F)):
        subarr = []
        for j in range(len(x1)):
            params1 = [*x1[0:j+1], *x2[j+1:len(x2)+1]]
            params2 = [*x1[0:j], *x2[j:len(x2)+1]]
            # print("x1[j] = {0}, x2[j] = {1}, x1[j] - x2[j] = {2}".format(x1[j], x2[j], x1[j] - x2[j]))
            # print((F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))))
            result = (F[i].subs(dict(zip(x, params1))) - F[i].subs(dict(zip(x, params2)))) / (x1[j] - x2[j])
            subarr.append(result)
        
        arr.append(subarr)
    
    return np.array(arr, dtype = 'float')


In [142]:
def NewthonMethod(x0, x, F):
    I = 0

    x1 = x0 - np.dot(np.linalg.inv(F_der_0(x0)), F_0(x0))
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I +=1
    
    # while max(np.abs(x1 - x0)) > eps:
    while True:
        x0 = x1
        x1 = x0 - np.dot(np.linalg.inv(F_der_0(x0)), F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1

        # x0 = x1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0

In [143]:
def PotraMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    I = 0

    matrix = (F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1)
    x1 = x0 - np.dot(np.linalg.inv(matrix), F_0(x0))
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
    I += 1

    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1

        matrix = (F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1)
        x1 = x0 - np.dot(np.linalg.inv(matrix), F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(I + 1, F_0(x1), x1))
        I += 1

        # x_2 = x_1
        # x_1 = x0
        # x0 = x1
    
    print('\n\nКількість ітерацій: {0}'.format(I))

    return x0


In [144]:
def PotraInverseOperatorMethod(x0, x, F):
    # element at -1
    x_1 = x0 - 10 ** -4

    #element at -2
    x_2 = x0 - 2 * (10 ** -4)

    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv((F_1(x0, x_1) + F_1(x_2, x0)) - F_1(x_2, x_1))

    x1 = x0 - np.dot(A_0, F_0(x0))
    matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
    # right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
    right_matrix = 2 * I - np.dot(matrix, A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
    number_of_iterations +=1
    
    while True:
        x_2 = x_1
        x_1 = x0
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        matrix = (F_1(x1, x0) + F_1(x_1, x1)) - F_1(x_1, x0)
        # right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
        right_matrix = 2 * I - np.dot(matrix, A_0)
        A_1 = np.dot(A_0, right_matrix)
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1

        # x_2 = x_1
        # x_1 = x0
        # x0 = x1
        # A_0 = A_1
    
    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0


In [145]:
# Inverse operator methods
def NewthonInverseOperatorMethod(x0, x, F):
    number_of_iterations = 0
    I = np.eye(len(F), dtype = 'float')
    A_0 = np.linalg.inv(F_der_0(x0))

    x1 = x0 - np.dot(A_0, F_0(x0))
    right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
    A_1 = np.dot(A_0, right_matrix)
    print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))

    number_of_iterations += 1
    
    # while max(np.abs(x1 - x0)) > eps:
    while True:
        x0 = x1
        A_0 = A_1

        x1 = x0 - np.dot(A_0, F_0(x0))

        if (max(np.abs(x1 - x0)) < eps):
            break

        right_matrix = 2 * I - np.dot(F_der_0(x1), A_0)
        A_1 = np.dot(A_0, right_matrix)
        
        print("{0} Ітерація: Значення F = {1} у точці x = {2}".format(number_of_iterations + 1, F_0(x1), x1))
        number_of_iterations += 1
        # x0 = x1
        # A_0 = A_1

    print('\n\nКількість ітерацій: {0}'.format(number_of_iterations))

    return x0
    

In [146]:
print("Метод Ньютона з послідовною апроксимацією оберненого оператора\n\n")
print(NewthonInverseOperatorMethod(x0, x, F))

Метод Ньютона з послідовною апроксимацією оберненого оператора


1 Ітерація: Значення F = [-2.22044605e-16  1.86190061e-16  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [0.         0.         0.09765625 1.23526471] у точці x = [ 0.74404762 -0.07440476  0.11904762  0.11904762]
3 Ітерація: Значення F = [0.         0.         0.04013928 0.50772621] у точці x = [ 0.47701881 -0.04770188  0.07632301  0.07632301]
4 Ітерація: Значення F = [5.55111512e-17 0.00000000e+00 1.67355214e-02 2.11689462e-01] у точці x = [ 0.30801391 -0.03080139  0.04928223  0.04928223]
5 Ітерація: Значення F = [0.         0.         0.00700849 0.08865119] у точці x = [ 0.19932556 -0.01993256  0.03189209  0.03189209]
6 Ітерація: Значення F = [-2.77555756e-17  0.00000000e+00  2.93912541e-03  3.71773225e-02] у точці x = [ 0.12908024 -0.01290802  0.02065284  0.02065284]
7 Ітерація: Значення F = [-1.38777878e-17  0.00000000e+00  1.23312296e-03  1.5597

In [147]:
print("Метод Потра з послідовною апроксимацією оберненого оператора\n\n")
print(PotraInverseOperatorMethod(x0, x, F))

Метод Потра з послідовною апроксимацією оберненого оператора


1 Ітерація: Значення F = [-8.14037726e-12  4.74164023e-14  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [2.64344102e-13 5.83395526e-15 9.76562500e-02 1.23526471e+00] у точці x = [ 0.74404762 -0.07440476  0.11904762  0.11904762]
3 Ітерація: Значення F = [5.47895063e-14 1.17920372e-15 4.01392812e-02 5.07726209e-01] у точці x = [ 0.47701881 -0.04770188  0.07632301  0.07632301]
4 Ітерація: Значення F = [1.21569421e-14 2.79285092e-16 1.67355214e-02 2.11689462e-01] у точці x = [ 0.30801391 -0.03080139  0.04928223  0.04928223]
5 Ітерація: Значення F = [2.74780199e-15 6.20633538e-17 7.00849198e-03 8.86511904e-02] у точці x = [ 0.19932556 -0.01993256  0.03189209  0.03189209]
6 Ітерація: Значення F = [6.10622664e-16 1.55158385e-17 2.93912541e-03 3.71773225e-02] у точці x = [ 0.12908024 -0.01290802  0.02065284  0.02065284]
7 Ітерація: Значення F = [1.66533454e-1

In [148]:
print("Метод Ньютона\n\n")
print(NewthonMethod(x0, x, F))

Метод Ньютона


1 Ітерація: Значення F = [-2.22044605e-16  1.86190061e-16  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [1.11022302e-16 0.00000000e+00 6.25000000e-02 7.90569415e-01] у точці x = [ 0.5952381  -0.05952381  0.0952381   0.0952381 ]
3 Ітерація: Значення F = [0.00000000e+00 1.55158385e-17 1.56250000e-02 1.97642354e-01] у точці x = [ 0.29761905 -0.0297619   0.04761905  0.04761905]
4 Ітерація: Значення F = [2.77555756e-17 0.00000000e+00 3.90625000e-03 4.94105884e-02] у точці x = [ 0.14880952 -0.01488095  0.02380952  0.02380952]
5 Ітерація: Значення F = [0.         0.         0.00097656 0.01235265] у точці x = [ 0.07440476 -0.00744048  0.01190476  0.01190476]
6 Ітерація: Значення F = [ 0.00000000e+00 -1.93947981e-18  2.44140625e-04  3.08816178e-03] у точці x = [ 0.03720238 -0.00372024  0.00595238  0.00595238]
7 Ітерація: Значення F = [0.00000000e+00 0.00000000e+00 6.10351563e-05 7.72040444e-04] у точці x =

In [149]:
print("Метод Потра\n\n")
print(PotraMethod(x0, x, F))

Метод Потра


1 Ітерація: Значення F = [-8.14037726e-12  4.74164023e-14  2.50000000e-01  3.16227766e+00] у точці x = [ 1.19047619 -0.11904762  0.19047619  0.19047619]
2 Ітерація: Значення F = [2.37909692e-12 2.33978844e-14 6.25000000e-02 7.90569415e-01] у точці x = [ 0.5952381  -0.05952381  0.0952381   0.0952381 ]
3 Ітерація: Значення F = [5.55111512e-17 0.00000000e+00 1.56250000e-02 1.97642354e-01] у точці x = [ 0.29761905 -0.0297619   0.04761905  0.04761905]
4 Ітерація: Значення F = [ 2.77555756e-17 -7.75791923e-18  3.90625000e-03  4.94105884e-02] у точці x = [ 0.14880952 -0.01488095  0.02380952  0.02380952]
5 Ітерація: Значення F = [-1.38777878e-17 -3.87895961e-18  9.76562500e-04  1.23526471e-02] у точці x = [ 0.07440476 -0.00744048  0.01190476  0.01190476]
6 Ітерація: Значення F = [6.93889390e-18 0.00000000e+00 2.44140625e-04 3.08816178e-03] у точці x = [ 0.03720238 -0.00372024  0.00595238  0.00595238]
7 Ітерація: Значення F = [ 0.00000000e+00 -9.69739904e-19  6.10351563e-05  7.720